# PRERREQUISITOS

**Este algoritmo solo funciona cuando tenemos los audios separados en cada canal, la voz del usuario en uno y la del agente en otro.**


In [1]:
# !pip install ibm-watson==4.5.0
# !pip install --upgrade "ibm-watson>=5.1.0"
#prefix = "C:Users/DavidX"
# import logging
# logging.basicConfig(level=logging.INFO , format='[%(levelname)s] (%(threadName)-s) %(message)s]')

  Created wheel for ibm-watson: filename=ibm_watson-5.1.0-py3-none-any.whl size=375445 sha256=7dd6afbc3ea8b32629e6392f72059075beacb8fcd2726ae496fc22bb55677321
  Stored in directory: c:\users\josbarlo\appdata\local\pip\cache\wheels\c1\40\da\ba44de639af32759a5755cb79afedf309104b13fea09bbb5fc
  Created wheel for ibm-cloud-sdk-core: filename=ibm_cloud_sdk_core-3.10.0-py3-none-any.whl size=60915 sha256=e28445f6aa39acc97cb26b750aefeadbe12276b997401f321f3dd0279d837a58
  Stored in directory: c:\users\josbarlo\appdata\local\pip\cache\wheels\09\6e\48\aa3801c63bbc5914ece2c66a6131925ca99a50d0a16355c8b5
Successfully built ibm-watson ibm-cloud-sdk-core
  Attempting uninstall: PyJWT
    Found existing installation: PyJWT 1.7.1
    Uninstalling PyJWT-1.7.1:
      Successfully uninstalled PyJWT-1.7.1
  Attempting uninstall: ibm-cloud-sdk-core
    Found existing installation: ibm-cloud-sdk-core 1.5.1
    Uninstalling ibm-cloud-sdk-core-1.5.1:
      Successfully uninstalled ibm-cloud-sdk-core-1.5.1
  Att

In [2]:
import numpy as np
import pandas as pd
import os
import json
from os.path import join, dirname
from ibm_watson import SpeechToTextV1
from ibm_watson.websocket import RecognizeCallback, AudioSource
from ibm_cloud_sdk_core.authenticators import IAMAuthenticator
import sqlite3 #repo de david para guardado de IBM

# WJu9N52Xfl4WWRAIVANKj7MYfZ4xuf6U0gXiOLp1bN3C
# https://api.us-south.speech-to-text.watson.cloud.ibm.com/instances/8ddbd4f4-074b-4ee4-9ac3-9b670e5151fe

# key_empresa='WJu9N52Xfl4WWRAIVANKj7MYfZ4xuf6U0gXiOLp1bN3C'
# url_empresa='https://api.us-south.speech-to-text.watson.cloud.ibm.com/instances/8ddbd4f4-074b-4ee4-9ac3-9b670e5151fe'



# key_empresa='DhHtT7uIEB3KGhAmlQQ1_fLVPkGERQ1l5QaoKZUGyLjt'
# url_empresa='https://stream.watsonplatform.net/speech-to-text/api'


# key_empresa = 'QEnZeaAyPM6UOvqhhr8CExb-iYgPzmbpVb_xfgzSx1a4'
# url_empresa = 'https://api.us-south.speech-to-text.watson.cloud.ibm.com/instances/317413e3-374f-4c4c-98d3-b320a149bdc6'

key_empresa = 'QEnZeaAyPM6UOvqhhr8CExb-iYgPzmbpVb_xfgzSx1a4'
url_empresa = 'https://api.us-south.speech-to-text.watson.cloud.ibm.com'



authenticator = IAMAuthenticator(key_empresa)
speech_to_text = SpeechToTextV1(
    authenticator=authenticator
)

speech_to_text.set_service_url(url_empresa)



# conexion a base de datos ##################################################################
para ='agente' #'usuario'####!!!!! modifico aqui


ingeniero_acargo = "andres_"
# ruta_db = "C:\\Users\\josbarlo\\Documents\\Confidencial\\Ecopetrol\\"+ingeniero_acargo+"salud.db" #MODIFICAR RUTA
ruta_db =   "Z:\\Ecopetrol\\data\\2021\\processed\\detallados\\Marzo_2021\\Marzo_per\\vol_ajust\\canales\\"+ingeniero_acargo+para+"_personal.db"
conn = sqlite3.connect(ruta_db)
table = """
CREATE TABLE IF NOT EXISTS speech_to_text (
	name_file  text,
	transcripcion text ,
	speech_ratio REAL,
	confidance REAL,
  owner text,
  raw json
);
"""

cur = conn.cursor()
cur.execute(table)
conn.commit()
conn.close()

DB_to_text = []


##################################################################################################

class MyRecognizeCallback(RecognizeCallback):

    name_raw_file = ""
    owner = ""

    def __init__(self):
        RecognizeCallback.__init__(self)


    def on_data(self, data):
        # logging.INFO("INICIO DE CONVERSION = "+self.name_raw_file)
        # conversion = json.dumps(data, indent=4)
        # print(conversion, self.name_raw_file)
       
        speech_ratio = data["audio_metrics"]["accumulated"]["speech_ratio"]
  
       
        promedio_confianza = []
        for indice, valueresult in enumerate(data["results"]):         
          for confianza in data["results"][indice]["alternatives"]:
            try:
              promedio_confianza.append(confianza["confidence"])
            except:
              promedio_confianza.append(0)  

        
        resultado_confianza = sum(promedio_confianza)/len(promedio_confianza)
        transcripcion = ""
        for transcrip in data["results"]:
          transcripcion = transcripcion+" "+transcrip["alternatives"][0]["transcript"]
        
                
        sql = ''' INSERT INTO speech_to_text(name_file,transcripcion,speech_ratio,confidance, owner,  raw)
          VALUES(?,?,?,?,?,? ) '''
        conn = sqlite3.connect(ruta_db)
        cur = conn.cursor()
        # print("name = ",self.name_raw_file)
        # print("trascripcion = ",transcripcion)
        # print("speech = ",speech_ratio)
        # print("confianza = ",resultado_confianza)
        # print("owner = ",self.owner)
        # print("json = ", json.dumps(data, indent=2))


        cur.execute(sql, [self.name_raw_file ,transcripcion,  speech_ratio, resultado_confianza, self.owner, json.dumps(data, indent=2)])
        print([self.name_raw_file ,transcripcion,  speech_ratio, resultado_confianza, self.owner, json.dumps(data, indent=2)])
        conn.commit()
        conn.close()


       



    def on_error(self, error):
        DB_to_text.append([self.name_raw_file, np.nan, np.nan, error])
        # print([self.name_raw_file ,transcripcion,  speech_ratio, resultado_confianza, self.owner, json.dumps(data, indent=2)],'on_error') 
        self.name_raw_file =""
        # print('Error received: {}'.format(error))
        return 

         

    def on_inactivity_timeout(self, error):
       DB_to_text.append([self.name_raw_file,np.nan, np.nan, error]) 
      #  print([self.name_raw_file ,transcripcion,  speech_ratio, resultado_confianza, self.owner, json.dumps(data, indent=2)],'on_inactivity') 
       self.name_raw_file = ""
       # print('Error received: {}'.format(error))
       
       return 

        



##################################################################################################

callbacks =[]
hilos = 10

for num in range(hilos):
   callbacks.append(MyRecognizeCallback())


def lectura_audio(path, myRecognizeCallback, owner):
  audio_file =open(path, 'rb')  
  myRecognizeCallback.name_raw_file = path
  myRecognizeCallback.owner = owner
  with audio_file:
    audio_source = AudioSource(audio_file)
    speech_to_text.recognize_using_websocket(
            audio=audio_source,
            audio_metrics=True,
            speaker_labels= False,
            timestamps=True,
            content_type='audio/wav',
            recognize_callback= myRecognizeCallback,
            model='es-ES_NarrowbandModel',
            inactivity_timeout= -1)
    audio_file.close()

In [3]:
ruta_db

'Z:\\Ecopetrol\\data\\2021\\processed\\detallados\\Marzo_2021\\Marzo_per\\vol_ajust\\canales\\andres_agente_personal.db'

# TEST

In [ ]:
# %%time
# import os
# os.chdir(r"C:/Users/andsalla/Documents/CONFIDENCIAL/SPEECH_ECOPETROL/FILE_AUDIO/Noviembre/test")  #IMPORTANTE DECIRLE AL NOTEBOOK DONDE ESTAN LOS ARCHIVOS

# lectura_audio("C:/Users/andsalla/Documents/CONFIDENCIAL/SPEECH_ECOPETROL/FILE_AUDIO/Noviembre/test/RQ21QO2HST63B28OIE5HQTGBLS002FL0_2020-11-03_14-56-54_2020-11-03_09-56-54_3185807130_28400_80248594-063B01FA-107BB882-00000001.wav",callbacks[0], "usuario" )


# PROCESAMIENTO

**Se debe elegir la ruta donde se encuentran almacenados los audios.**


In [4]:
#cambiar mes y usuario o agente
import os
# os.chdir(r"C:\Users\josbarlo\Documents\Confidencial\Ecopetrol\mono") #IMPORTANTE DECIRLE AL NOTEBOOK DONDE ESTAN LOS ARCHIVOS
# os.chdir(r'Z:\Ecopetrol\data\2021\processed\detallados\Marzo_2021\Marzo_per\vol_ajust\canales\canal_agente')
os.chdir(r'Z:\Ecopetrol\data\2021\processed\detallados\Marzo_2021\Marzo_per\vol_ajust\canales\canal_'+para)
files = os.listdir()
len(files)

487

In [5]:
os.getcwd()

'Z:\\Ecopetrol\\data\\2021\\processed\\detallados\\Marzo_2021\\Marzo_per\\vol_ajust\\canales\\canal_agente'

In [6]:
#Cuando se está procesando la interlocución del agente, colocar en files["owner"] = "agente"; de lo contrario files["owner"] = "usuario"
files = pd.DataFrame(files, columns=["name"])
files["owner"] = para
files

,name,owner
0,IVBB17DJ512BJD7SVU27C4AFC8003LI2_2021-03-01_12...,agente
1,IVBB17DJ512BJD7SVU27C4AFC8003LI2_2021-03-01_12...,agente
2,IVBB17DJ512BJD7SVU27C4AFC8003LJ1_2021-03-01_12...,agente
3,IVBB17DJ512BJD7SVU27C4AFC8003LK3_2021-03-01_13...,agente
4,IVBB17DJ512BJD7SVU27C4AFC8003LKR_2021-03-01_13...,agente
...,...,...
482,O7BIVFNJS52B52NDGEM97G3J5G00062I_2021-03-31_19...,agente
483,O7BIVFNJS52B52NDGEM97G3J5G000696_2021-03-31_19...,agente
484,O7BIVFNJS52B52NDGEM97G3J5G00069K_2021-03-31_19...,agente
485,O7BIVFNJS52B52NDGEM97G3J5G00069K_2021-03-31_19...,agente


In [7]:
files.iloc[0]['name']

'IVBB17DJ512BJD7SVU27C4AFC8003LI2_2021-03-01_12-39-21_2021-03-01_07-39-22_14300654_28400_1081409330-063E0203-1094F8AD-00000001.wav'

In [8]:
files.iloc[1]['name']


'IVBB17DJ512BJD7SVU27C4AFC8003LI2_2021-03-01_12-45-19_2021-03-01_07-45-19_14300654_28400_1098795247-063A0203-10959D4A-00000001.wav'

**PROCESO FINALIZADO...**


In [ ]:
# #Solo si se deja procesando una parte
# conn = sqlite3.connect("/Users/Usuario/Documents/Audios_Ecopetrol/DATA/FILE_AUDIO/julio/speech_ecopetrol_natalia_repro.db")
# sql = "SELECT * FROM speech_to_text;"

# a=files.merge(pd.read_sql(sql,conn), right_on="name_file" , left_on="name", how="left") 
# files=a[a.name_file.isna()]
# files=files[['name','owner_x']]
# files=files.rename(columns={'owner_x':'owner'})
# files.shape

In [ ]:
# ###PARA REPROCESAR 
# files=pd.read_csv('/Users/Usuario/Documents/Audios_Ecopetrol/DATA/FILE_AUDIO/julio/reprocesar_julio.csv')
# files=files.rename(columns={'name_file':'name'})
# files=files[files.owner=='usuario']
# files.shape

In [9]:
files

,name,owner
0,IVBB17DJ512BJD7SVU27C4AFC8003LI2_2021-03-01_12...,agente
1,IVBB17DJ512BJD7SVU27C4AFC8003LI2_2021-03-01_12...,agente
2,IVBB17DJ512BJD7SVU27C4AFC8003LJ1_2021-03-01_12...,agente
3,IVBB17DJ512BJD7SVU27C4AFC8003LK3_2021-03-01_13...,agente
4,IVBB17DJ512BJD7SVU27C4AFC8003LKR_2021-03-01_13...,agente
...,...,...
482,O7BIVFNJS52B52NDGEM97G3J5G00062I_2021-03-31_19...,agente
483,O7BIVFNJS52B52NDGEM97G3J5G000696_2021-03-31_19...,agente
484,O7BIVFNJS52B52NDGEM97G3J5G00069K_2021-03-31_19...,agente
485,O7BIVFNJS52B52NDGEM97G3J5G00069K_2021-03-31_19...,agente


In [10]:
temo = files.copy()
len(temo)

487

In [11]:
%%time
import threading
from tqdm import tqdm
from IPython.display import clear_output


inicio= 0
catidad_registros = len(files)
intervalo = int(catidad_registros/5) 


def task1():    
    for indice , row in tqdm(files[inicio:intervalo].iterrows(), desc="hilo_1", ncols=47):
      lectura_audio(row["name"], callbacks[0], row["owner"])

def task2():
  for indice, row in tqdm(files[intervalo:intervalo*2].iterrows(), desc="hilo_2", ncols=47):
    lectura_audio(row["name"], callbacks[1], row["owner"])


def task3():    
    for indice, row in tqdm(files[intervalo*2:intervalo*3].iterrows(), desc="hilo_3", ncols=47):
      lectura_audio(row["name"], callbacks[2], row["owner"])

def task4():    
    for indice,  row in tqdm(files[intervalo*3:intervalo*4].iterrows(), desc="hilo_4", ncols=47):
      lectura_audio(row["name"], callbacks[3], row["owner"])

def task5():    
    for indice, row in tqdm(files[intervalo*4:].iterrows(), desc="hilo_5", ncols=47):
      lectura_audio(row["name"], callbacks[4], row["owner"])





hilo1 = threading.Thread(name='hilo1', 
                            target=task1)    
hilo2 = threading.Thread(name='hilo2' , 
                            target=task2)    
hilo3 = threading.Thread(name='hilo3', 
                            target=task3)    

hilo4 = threading.Thread(name='hilo4', 
                            target=task4)  
hilo5 = threading.Thread(name='hilo5', 
                            target=task5)  

hilo1.start()
hilo2.start()
hilo3.start()
hilo4.start()
hilo5.start()

hilo1.join()
print("hilo_esperando")
hilo2.join()
print("hilo2_esperando")
hilo3.join()
print("hilo3_esperando")
hilo4.join()
print("hilo4_esperando")

hilo5.join()
print("hilo5_esperando")


hilo_1: 0it [00:00, ?it/s]
hilo_4: 0it [00:00, ?it/s]

hilo_3: 0it [00:00, ?it/s]


hilo_5: 0it [00:00, ?it/s]



hilo_2: 0it [00:00, ?it/s]


hilo_5: 1it [00:01,  1.78s/it]

hilo_3: 1it [00:01,  1.81s/it]
hilo_4: 1it [00:01,  1.82s/it]



hilo_1: 1it [00:02,  2.11s/it]
hilo_4: 2it [00:02,  1.44s/it]

hilo_3: 2it [00:02,  1.44s/it]


hilo_1: 2it [00:02,  1.65s/it]
hilo_4: 3it [00:02,  1.11s/it]



hilo_2: 2it [00:02,  1.62s/it]


hilo_5: 3it [00:02,  1.14s/it]
hilo_1: 3it [00:02,  1.22s/it]

hilo_3: 3it [00:02,  1.17s/it]


hilo_5: 4it [00:03,  1.17it/s]



hilo_1: 4it [00:03,  1.08it/s]

hilo_3: 4it [00:03,  1.12it/s]
hilo_4: 5it [00:03,  1.50it/s]



hilo_1: 5it [00:03,  1.40it/s]
hilo_4: 6it [00:03,  1.90it/s]

hilo_3: 5it [00:03,  1.43it/s]


hilo_1: 6it [00:03,  1.77it/s]
hilo_4: 7it [00:03,  2.31it/s]



hilo_2: 5it [00:03,  1.40it/s]

hilo_3: 6it [00:03,  1.84it/s]


hilo_5: 6it [00:03,  1.76it/s]
hilo_1: 7it [00:03,  2.19it/s]



hilo_2: 6it [00:03,  1.74it/s]

hilo_3: 7it [00:

hilo_esperando


hilo_4: 87it [00:25,  3.91it/s]



hilo_2: 87it [00:25,  3.45it/s]

hilo_3: 82it [00:25,  3.38it/s]


hilo_5: 91it [00:25,  3.11it/s]
hilo_4: 88it [00:26,  4.07it/s]



hilo_2: 88it [00:26,  3.68it/s]


hilo_5: 92it [00:26,  3.45it/s]

hilo_3: 83it [00:26,  3.44it/s]
hilo_4: 89it [00:26,  4.05it/s]



hilo_2: 89it [00:26,  3.74it/s]


hilo_5: 93it [00:26,  3.36it/s]

hilo_3: 84it [00:26,  3.24it/s]
hilo_4: 90it [00:26,  4.33it/s]



hilo_2: 90it [00:26,  3.92it/s]


hilo_5: 94it [00:26,  3.67it/s]
hilo_4: 91it [00:26,  4.64it/s]

hilo_3: 85it [00:26,  3.45it/s]



hilo_2: 91it [00:26,  4.03it/s]
hilo_4: 92it [00:26,  4.90it/s]


hilo_5: 95it [00:26,  3.67it/s]

hilo_3: 86it [00:26,  3.64it/s]



hilo_2: 92it [00:27,  3.58it/s]


hilo_5: 96it [00:27,  3.65it/s]

hilo_3: 87it [00:27,  3.50it/s]
hilo_4: 93it [00:27,  3.51it/s]



hilo_2: 93it [00:27,  3.72it/s]

hilo_3: 88it [00:27,  3.57it/s]
hilo_4: 94it [00:27,  3.62it/s]


hilo_5: 97it [00:27,  3.10it/s]



hilo_2: 94it [00:27,  3.74i

hilo2_esperando




hilo_3: 92it [00:29,  2.61it/s]

hilo_3: 93it [00:29,  2.91it/s]

hilo_3: 94it [00:29,  2.67it/s]

hilo_3: 95it [00:30,  2.86it/s]

hilo_3: 96it [00:30,  2.87it/s]

hilo_3: 97it [00:30,  3.15it/s]

hilo3_esperando
hilo4_esperando
hilo5_esperando
Wall time: 31.2 s


In [ ]:
files.head()

,name,owner
0,IVBB17DJ512BJD7SVU27C4AFC8003LI2_2021-03-01_12...,agente
1,IVBB17DJ512BJD7SVU27C4AFC8003LI2_2021-03-01_12...,agente
2,IVBB17DJ512BJD7SVU27C4AFC8003LJ1_2021-03-01_12...,agente
3,IVBB17DJ512BJD7SVU27C4AFC8003LK3_2021-03-01_13...,agente
4,IVBB17DJ512BJD7SVU27C4AFC8003LKR_2021-03-01_13...,agente


In [ ]:
len(files)

487